In [ ]:
pip install contractions

In [ ]:
import pandas as pd
import boto3, sys,sagemaker                             
import pandas as pd   
import pandas as pd
import numpy as np
import nltk
import string
import contractions
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
# plt.xticks(rotation=70)
pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', 100)
%matplotlib inline
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [ ]:
df = pd.read_csv('cleanedText.csv', index_col=0)

In [ ]:
df['no_punc'][1]

In [ ]:
table = df.loc[:,['TEXT']]

In [ ]:
nltk.download('punkt')
table['tokenizedSent'] = table['TEXT'].apply(sent_tokenize)


In [ ]:
table['tokenizedSent'][1]

In [ ]:
table['tokenized_words'] = table['tokenizedSent'].apply(lambda x: [word_tokenize(sent) for sent in x])
# punc = string.punctuation
# table['no_punc/numbers'] = table['tokenizedSent'].apply(lambda x: [word.lower() for word in x if word not in punc])

In [ ]:
table['tokenized_words'][1]

In [ ]:
 table['no_punc/numbers'] = table['tokenized_words'].apply(lambda x: [[word.lower() for word in sent if word not in punc and word.isalpha()] for sent in x])

In [ ]:
 table['no_punc/numbers'][1]

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
table['stopwords_removed'] = table['no_punc/numbers'].apply(lambda x: [[word for word in sent if word not in stop_words]for sent in x])

In [ ]:
table['stopwords_removed'][1]

In [ ]:
nltk.download('averaged_perceptron_tagger')
table['pos_tags'] = table['stopwords_removed'].apply(lambda x: [nltk.tag.pos_tag(sent) for sent in x if len(sent)> 0 ])

In [ ]:
table['pos_tags'][1]

In [ ]:
nltk.download('wordnet')
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
table['wordnet_pos'] = table['pos_tags'].apply(lambda x: [[(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in sent]for sent in x])

In [ ]:
table['wordnet_pos'][1]

In [ ]:
wnl = WordNetLemmatizer()
table['lemmatized'] = table['wordnet_pos'].apply(lambda x: [[wnl.lemmatize(word, tag) for word, tag in sent] for sent in x])


In [ ]:
table.head()

In [ ]:
table.to_csv('sentenceRowClean.csv')

In [ ]:
pip install gensim

In [ ]:
import logging
from gensim.models.word2vec import Word2Vec
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
w2v_model2 = Word2Vec(table['lemmatized'], vector_size=100, min_count=2)

In [ ]:
y = list(table['lemmatized'])
sentences = []
for row in y:
    for sent in row:
        sentences.append(sent)
        

In [ ]:
import logging
from gensim.models.word2vec import Word2Vec
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
w2v_model2 = Word2Vec(sentences, vector_size=100, min_count=2)

In [ ]:
w2v_model2.wv.most_similar(['disease'])

In [ ]:
w2v_model2.wv.most_similar(['cancer'])

In [ ]:
w2v_model2.save('w2vec2')

In [ ]:
model1[w]